# 1. Imports

In [5]:
# our functions
import functions as f
# readers

import gzip
import json

import codecs

ModuleNotFoundError: No module named 'vectorizer'

In [49]:
PATH = {}
PATH["dataset_classification"] = "dataset/classification/"
PATH["dataset_labeling"] = "dataset/seq_labeling/"
PATH["music_reviews_train"] = PATH["dataset_classification"] + "music_reviews_train.json.gz"
PATH["music_reviews_dev"] = PATH["dataset_classification"] + "music_reviews_dev.json.gz"
PATH["music_reviews_test"] = PATH["dataset_classification"] + "music_reviews_test_masked.json.gz"

# 2. Read in the files

In [50]:
train = f.readJson(PATH["music_reviews_train"])
dev = f.readJson(PATH["music_reviews_dev"])
test = f.readJson(PATH["music_reviews_test"])

Number of data:  100000
Number of data:  10000
Number of data:  10000


In [77]:
data = train
train_sent = []
train_sentiment = []
train_idx = []
missing_indexies = []
for i in range(len(data)):
    try:
        train_sent.append(data[i]["reviewText"])
        train_sentiment.append(data[i]["sentiment"])
        train_idx.append(i)
    except KeyError:
        missing_indexies.append(i)
        continue
print(missing_indexies)

[4603, 4838, 16019, 18229, 19785, 23315, 28397, 28834, 33359, 43138, 43539, 43690, 44008, 44477, 44972, 48811, 49317, 50106, 51717, 52286, 55555, 56171, 57223, 58328, 58799, 58866, 59525, 59739, 61046, 61914, 61916, 62831, 63208, 72268, 78944, 79067, 80093, 80637, 80658, 81640, 81900, 82510, 83138, 83145, 83615, 84761, 87112, 88895, 88984, 89132, 91949, 94301, 94727, 99641]


In [78]:
data = test
test_sent = []
test_sentiment = []
test_idx = []
missing_indexies = []
for i in range(len(data)):
    try:
        test_sent.append(data[i]["reviewText"])
        test_sentiment.append(data[i]["sentiment"])
        test_idx.append(i)
    except KeyError:
        missing_indexies.append(i)
        continue
print(missing_indexies)

[90, 117, 439, 2138, 4112, 5364, 8210]


In [80]:
from zeugma import TextsToSequences
sequencer = TextsToSequences()
sequencer.fit_transform(test_sent)
#print(sequencer.index_word)
# fit new data 
Train = sequencer.transform(test_sent)
Test = sequencer.transform(test_sent)

In [81]:
Test

array([list([292]), list([108, 181]), list([27, 52, 36]), ...,
       list([6143, 11229, 58, 5515, 15, 6, 22, 5, 31, 1147, 522, 643, 30, 58, 38, 417, 387, 3, 11102, 58, 730, 1119, 1477, 12, 47, 3074, 2208, 18, 38, 289, 54, 59, 2, 152, 2208, 2, 75, 14, 694, 6, 8, 378, 49, 23214, 98, 1, 38, 289, 98, 20, 4, 1587, 23215, 1495, 25, 47, 1348, 2711, 6143, 10839, 191, 10, 58, 702, 22, 6, 8, 48, 16, 64, 3, 1920, 125, 1, 2208, 6627, 219, 6021, 23216, 9405, 90, 44, 1, 264, 1187, 6, 22, 11229, 13, 54, 3, 738, 3603, 5, 58, 54, 9, 8, 21, 136, 6143, 22, 1224, 5038, 31, 1, 273, 29, 2, 1603, 12, 202, 14, 675, 20, 6143, 3, 88, 26, 6, 284, 7, 19, 16, 25, 10, 12, 4, 1424, 23217, 46, 13, 4931, 121, 7, 23218, 7132, 150, 5771, 4110, 337, 278, 1121, 2831, 995, 10364, 3, 2681, 57, 25, 128, 2415, 2764, 7, 23219, 229, 2, 53, 14, 152, 431, 253, 1955, 1, 86, 298, 81, 4, 170, 299, 7, 11230, 11230, 1105, 18, 14, 32, 23220, 1955, 1412, 7, 1, 2239, 4959, 4399, 3, 23, 40, 16, 142, 428, 102, 109, 60, 492, 229, 34, 1, 10

In [63]:
raw[1000]

[2,
 2216,
 66,
 12,
 43,
 12515,
 5,
 160,
 48,
 2,
 165,
 18,
 59,
 56,
 631,
 2844,
 3693,
 2123,
 15,
 1,
 684,
 678,
 7,
 2655,
 3,
 125,
 5,
 64,
 1658,
 5,
 274,
 58,
 6589,
 138,
 2,
 17,
 12516,
 32,
 2,
 12517,
 125,
 2787,
 3,
 1474]

In [62]:
sent[1000]

'I signed up for his newsletter to see what I thought but when he started sending email info on the Dark Art of Hypnosis and how to get women to take their clothes off I was sickened so I unsubsbcribed.  How creepy and sick.'

In [61]:
print(sequencer.index_word)

{1: 'the', 2: 'i', 3: 'and', 4: 'a', 5: 'to', 6: 'this', 7: 'of', 8: 'is', 9: 'it', 10: 'in', 11: 'song', 12: 'for', 13: 'that', 14: 'not', 15: 'on', 16: 'you', 17: 'was', 18: 'but', 19: 'music', 20: 'with', 21: 'my', 22: 'album', 23: 'as', 24: 'have', 25: 'are', 26: 'like', 27: 'great', 28: 'love', 29: 'one', 30: 'from', 31: 'be', 32: 'so', 33: 'songs', 34: 'all', 35: 'just', 36: 'good', 37: 'by', 38: 'very', 39: 'at', 40: 'if', 41: 'me', 42: 'cd', 43: 'his', 44: "it's", 45: 'they', 46: 'has', 47: 'an', 48: 'what', 49: 'or', 50: 'more', 51: 'out', 52: 'really', 53: 'would', 54: 'sound', 55: 'will', 56: 'he', 57: 'there', 58: 'their', 59: 'when', 60: 'some', 61: 'can', 62: 'version', 63: 'no', 64: 'get', 65: 'her', 66: 'up', 67: 'listen', 68: 'time', 69: "don't", 70: 'only', 71: 'your', 72: 'these', 73: 'first', 74: 'had', 75: 'do', 76: 'about', 77: 'original', 78: 'well', 79: 'track', 80: 'much', 81: 'even', 82: 'band', 83: 'heard', 84: 'best', 85: 'been', 86: 'other', 87: 'which', 88

In [9]:
tiny_corpus = []
tiny_corpus.append(train[0]["reviewText"])
tiny_corpus.append(train[1]["reviewText"]) 
tiny_corpus
r = f.skip_gram(tiny_corpus)

In [11]:
print(r[0]) # labels

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9,  8, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 23, 33, 34,
        35, 36, 37, 38, 39,  3, 40, 41, 42])


In [12]:
print(r[1]) # skip_grams

tensor([[ 0,  0,  2,  3],
        [ 0,  1,  3,  4],
        [ 1,  2,  4,  5],
        [ 2,  3,  5,  6],
        [ 3,  4,  6,  7],
        [ 4,  5,  7,  8],
        [ 5,  6,  8,  9],
        [ 6,  7,  9,  8],
        [ 7,  8,  8, 10],
        [ 8,  9, 10, 11],
        [ 9,  8, 11, 12],
        [ 8, 10, 12, 13],
        [10, 11, 13, 14],
        [11, 12, 14, 15],
        [12, 13, 15, 16],
        [13, 14, 16, 17],
        [14, 15, 17, 18],
        [15, 16, 18, 19],
        [16, 17, 19, 20],
        [17, 18, 20, 21],
        [18, 19, 21, 22],
        [19, 20, 22, 23],
        [20, 21, 23, 24],
        [21, 22, 24, 25],
        [22, 23, 25,  0],
        [23, 24,  0,  0],
        [ 0,  0, 27, 28],
        [ 0, 26, 28, 29],
        [26, 27, 29, 30],
        [27, 28, 30, 31],
        [28, 29, 31, 32],
        [29, 30, 32, 23],
        [30, 31, 23, 33],
        [31, 32, 33, 34],
        [32, 23, 34, 35],
        [23, 33, 35, 36],
        [33, 34, 36, 37],
        [34, 35, 37, 38],
        [35,

In [ ]:
# viberti